In [ ]:
### uncomment to display figures
%matplotlib inline   

# Using the AWRA MS to run a Simulation of the AWRA-L model
There are two ways to run a simulation of the AWRA-L model in the AWRA MS
* Using the On-Demand Simulator
* Using the Server Simulator

This notebook outlines how to run a basic on-demand AWRA-L simulation and get into more detail about how to change the model configuration (nodegraph) to define the inputs to the simulation.

## On-Demand Simulation
The AWRA MS On-Demand Simulator is designed to run the model for a few years over a small extent. It allows the user to quickly and efficiently assess the impact of changes made to the model or inputs without the need to write all of the outputs to file. The user can write the results out but it is generally designed to hold the results in memory for visualisation and checking.<br><br>

This notebook outlines a basic On-Demand Simulation of the AWRA-L model and go through some of the options and functionality available from the package to modify a model run and inspect the outputs.


This notebook goes through the following steps:

1. Import required libraries
2. Quick example: the default model run<br>
3. Modifying the model configuration <br>
 3.1 Change forcing data<br>
 3.2 Change spatial grids<br>
 3.3 Change parameter values<br>
4. Put model run specification together<br>
 4.1 Instantiate the simulator<br>
 4.2 Specify period and extents<br>
 4.3 Run the model<br>
5. Configuring model outputs<br>
 5.1 Check what outputs are available<br>
 5.2 Add new output<br>
 5.3 Save outputs to files<br>
6. More examples<br>
 6.1 Run over entire continent<br>
 6.2 Run with uniform rain<br>
7. Exercise

### 1. Import required libraries

In [ ]:
## External Python packages

import os
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
## AWRAMS utilities
from awrams.utils import extents
from awrams.utils import datetools as dt

In [ ]:
## AWRAMS input nodegraph. The nodegraph is created when building the input mapping
from awrams.utils.nodegraph import nodes

In [ ]:
## AWRAMS parameter utilities
from awrams.utils import parameters

In [ ]:
## Select simulation option
from awrams.simulation import ondemand


In [ ]:
## Import the config manager - we'll use this a lot!
from awrams.utils import config_manager

### 2. Quick example: run the default model and settings

Quick example to show you where we are heading

In [ ]:
# The model profile is our entry point into obtaining and configuring information needed for a model run

model_profile = config_manager.get_model_profile('awral','v6_default')

# The system profile contains non-model-specific information related to the system we are running on, like file paths
# The following is equivalent to 
# sys_profile = config_manager.get_system_profile('default')

sys_profile = config_manager.get_system_profile()

# Settings are dictionaries of configurable options - you use profiles to access these

sys_settings = sys_profile.get_settings()

# Model settings uses the system settings dictionary to fill out its paths etc...
# You can use defaults, or pass in a sys_settings dict

model_settings = model_profile.get_settings(sys_settings)


In [ ]:
# The input mapping contains all the data related input configuration for a run (files, parameters, transforms...)

input_map = model_profile.get_input_mapping()

# If you want to use custom model settings, you can pass this in manually - more on this soon...

input_map = model_profile.get_input_mapping(model_settings)


In [ ]:
# The model object represents the actual runnable AWRA-L model, rather than abstract configuration information
# Again, this uses the model_settings dictionary; if you leave this argument out it will use the defaults

model = model_profile.get_model(model_settings)

In [ ]:
sim = ondemand.OnDemandSimulator(model, input_map) # Define a simulator with available params
results = sim.run(dt.dates('1 Jan 2010', '2 Jan 2010'), extents.get_default_extent()) # Runs over the whole country for one day

### 3. Modifying the model configuration


#### 3.1 Examine the  default input nodegraph

In [ ]:
# Get our default input map again

input_map = model_profile.get_input_mapping()

In [ ]:
# What's in the map?

list(input_map)

In [ ]:
input_map['hveg_hrusr']

Changing the inputs to the model can be done by operating on the input map. <br> This can be done by changing properties of the existing items, inserting new values, or by using functions that transform the map

#### 3.1 Change forcing inputs

In [ ]:
# The system settings dictionary contains several preconfigured climate datasets
# Have a look at the default config file for examples of how these are laid out; this is a useful template if you
# want to add your own later on

print(list(sys_settings.CLIMATE_DATASETS))

In [ ]:
# Let's have a look at the training dataset

sys_settings.CLIMATE_DATASETS['TRAINING']

In [ ]:
# As you can see, this is identical to the model defaults
# This is because we used sys_settings.CLIMATE_DATASETS['TRAINING'] in the default model config; ie there is no
# need to manually copy these paths, just refer to the existing dataset

model_settings.CLIMATE_DATASET

In [ ]:
# You can see these reflected in the forcing items of the input map; these are appended with '_f'
# to indicate that the forcing data is sourced from a file (this is just a convention, but worth remembering)

input_map['tmax_f']

In [ ]:
# It's easy to change datasets to any of the defaults, or you can construct your own as a dictionary

model_settings.CLIMATE_DATASET = sys_settings.CLIMATE_DATASETS['TESTING']

# Remember to regenerate the input map using your updated settings...

input_map = model_profile.get_input_mapping(model_settings)

# The forcing nodes reflect the updated paths

input_map['tmax_f']

In [ ]:
# Alternatively, you can operate directly on the input map itself.  This is convenient for quickly pointing to new data
# without editing config files

input_map['tmax_f'] = nodes.forcing_from_ncfiles(model_settings.CLIMATE_DATASET.FORCING.PATH,\
                                                 model_settings.CLIMATE_DATASET.FORCING.MAPPING['tmax'][0],
                                                model_settings.CLIMATE_DATASET.FORCING.MAPPING['tmax'][1])

#### 3.2 Changing the spatial input grids

In [ ]:
model_settings.SPATIAL_FILE

In [ ]:
import h5py

h = h5py.File(model_settings.SPATIAL_FILE,'r')

In [ ]:
list(h['parameters'].keys())

In [ ]:
kdsat_grid = h['parameters']['kdsat'][:]

In [ ]:
im = plt.imshow(kdsat_grid) 
plt.title("kdsat")
plt.colorbar(im)

#Don't panic at the disco effect. Grids have been infilled to cater for potential edge effects

In [ ]:
# Couple of ways you can look at the mapping for a particular grid
input_map['kdsat_grid']

In [ ]:
# If you want to modify any of the arguments
input_map.kdsat_grid.args

In [ ]:
# If you want to modify any of the arguments
input_map.kdsat_grid.args['filename']

In [ ]:
# The nodes library allows you to point to different grid file (.nc, .flt, anything recognised by gdal) to load up the data
# input_map.f_tree_grid = nodes.spatial_from_file(PATH_TO_NEW_FTREE_FILE)

#### 3.3 How to change a parameter

In [ ]:
# In most cases there are convenience functions for dealing with common tasks
# Use the parameters module we imported from awrams.utils earlier;
# The following Dataframe uses a standard layout than many parts of AWRAMS understands

param_df = parameters.input_map_to_param_df(input_map)

param_df

In [ ]:
# Change some values in the DataFrame

# Set the value of a single parameter
param_df.set_value('ssmax_scale','value',2.1)

# Change the 'fixed' property to True - this means the calibration system will now use 
# this value directly ie. it is fixed rather than calibrated

param_df.set_value('ssmax_scale','fixed',True)

param_df.loc['ssmax_scale']

In [ ]:
# So far we've just been modifying a DataFrame; update the input map to tell AWRAMS to use the new values

input_map = parameters.param_df_to_mapping(param_df,input_map)

input_map.ssmax_scale

In [ ]:
# For simple changes, it may be easier to operate directly on the mapping..

input_map.ssmax_scale.args

In [ ]:
input_map.ssmax_scale.args.value = 1.95

### 4. Put model run specification together

Like we did at the start for the default version

#### 4.1	Instantiate the simulator

In [ ]:
sim = ondemand.OnDemandSimulator(model,input_map)

#### 4.2	Define the required period and spatial extent

In [ ]:
period = dt.dates('dec 2010 - jan 2011')

In [ ]:
## Set the starting extent as the extent of the AWAP grid
## in the background this picks up the geospatial references associated with the grid set as default in the configuration file 
## Config.py ?

extent_default = extents.get_default_extent()

In [ ]:
## Select a sub-area of that grid, say the Perth region [450 grid cells south, 20 grid cells across]

extent = extent_default.ioffset[400:450,50:100]

In [ ]:
extent

#### 4.3 Run the model for the defined extent and period

In [ ]:
results = sim.run(period, extent)

A switch is available in the model run command that allows the process to capture the inputs, i,e.  forcing data and grid values that apply specifically to the extent run.


In [ ]:
## 4.3.1 Option to capture model inputs

results, inputs = sim.run(period, extent, return_inputs = True) # results holds  model outputs, inputs holds model gridded inputs/parameters

In [ ]:
## Have a look at some forcing input data
forcing_keys = ['tmin_f','tmax_f','precip_f','wind_f']

climate_inputs = {k:inputs[k] for k in forcing_keys}

list(climate_inputs)

In [ ]:
climate_inputs['precip_f'].shape

In [ ]:
im = plt.imshow(climate_inputs['tmin_f'][0])
plt.colorbar(im)

### 5. Configuring model outputs

#### 5.1 Check what outputs are available

Should be as per what's in the default output mapping + an extra one for the model states captured to be able to hotstart

In [ ]:
list(results) # this is to see what's in the model run outputs

In [ ]:
results['ss_hrusr']

In [ ]:
# Examine one of the cells (30,30) over the whole modelling period
# We'll use Pandas to make this easier...

df = pd.DataFrame(index=period)

# Examine the shallow soil layers for both HRUs
df['ss_hrusr'] = results['ss_hrusr'][:,30,30]
df['ss_hrudr'] = results['ss_hrudr'][:,30,30]

# Include some input data
df['precip_f'] = inputs['precip_f'][:,30,30]

df.plot()

#### 5.2 Add extra outputs to the model

In [ ]:
# See the currently selected set of model outputs
model_settings.OUTPUTS

In [ ]:
# Turn on the individual HRU outputs for ifs (interflow for the shallow soil layer)

model_settings.OUTPUTS['OUTPUTS_HRU'].append('ifs')

In [ ]:
# Now add ifs to OUTPUTS_AVG; the area-weighted average of ifs across both HRUs

model_settings.OUTPUTS['OUTPUTS_AVG'].append('ifs')

In [ ]:
# Obtain a new model object with the updated settings

model = model_profile.get_model(model_settings)

In [ ]:
## Run a simulation with the new outputs

sim = ondemand.OnDemandSimulator(model, input_map)
results = sim.run(period, extent)

In [ ]:
results['ifs'].shape

In [ ]:
df = pd.DataFrame(index=period)

df['ifs_hrusr'] = results['ifs_hrusr'][:,30,30]
df['ifs_hrudr'] = results['ifs_hrudr'][:,30,30]
df['ifs_avg'] = results['ifs'][:,30,30]

df.plot()

#### 5.3 Save outputs to file

By default, data is generated only in memory, not written out to files. <br>
Typically you would use the SimulationServer for this purpose, however it is still possible to do so with the OnDemandSimulator  

In [ ]:
from awrams.simulation.support import build_output_mapping

In [ ]:
## Re-run

In [ ]:
# See what data the model is outputting (in memory)

model.get_output_variables()

In [ ]:
# We probably don't want _all_ those written out
save_vars = ['qtot','s0_hrusr','s0_hrudr']

# Set a path to write to
outpath = './_results/'

output_map = build_output_mapping(model, outpath, save_vars = save_vars)

In [ ]:
sim_with_outputs = ondemand.OnDemandSimulator(model, input_map, output_map)

In [ ]:
period = dt.dates('dec 2010 - jan 2011')
results, iresults = sim_with_outputs.run(period, extent, True)

In [ ]:
# Files generated by this run...

os.listdir('./_results')

Additional things you can do with the model configuration, such as changing initial states and infilling gaps in the forcing inputs are presented in the [SimulationServer] notebook

[SimulationServer]: ./SimulationServer.ipynb


### 6. More examples

#### 6.1 Run over the entire continent. 
Just for one day. Likely to have memory issues if want to run for long periods.
2MB per output variable per day.

In [ ]:
period = dt.dates('jan 1 2011')
results, inputs = sim.run(period,extent_default,True)

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = [12.,8.]

In [ ]:
im = plt.imshow(results['qtot'][0],interpolation='None')
plt.colorbar(im)

In [ ]:
# Also view slope gridded input

im = plt.imshow(inputs['slope'],interpolation='None')
plt.colorbar(im)

In [ ]:
# Grid cell elevation range (highest point of hypsometric curve  - lowest point)

im = plt.imshow(inputs['height'][-1]-inputs['height'][0],interpolation='None')
plt.colorbar(im)

In [ ]:
# Rainfall input on the first day of simulation

im = plt.imshow(inputs['pt'][0],interpolation='None')
plt.colorbar(im)

#### 7.2 Run with a uniform rain input across the country

In [ ]:
input_map.pt = nodes.const(1000)

In [ ]:
runner = ondemand.OnDemandSimulator(model, input_map)

In [ ]:
results_fixedpt = runner.run(period, extent_default)

In [ ]:
im = plt.imshow(results_fixedpt['qtot'][0],interpolation='None')
plt.colorbar(im)

### 8. Exercise

1. Run the model over the same catchment multiple times, each time with a different parameter value for a parameter of your choice. Then plot the parameter values vs the average flow over the modelled period.